In [3]:
import os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import cv2
from PIL import Image
import h5py

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17069979618880343513
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4158062592
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6561061539952742558
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [17]:
!mkdir RetinanetTutorial

In [21]:
os.chdir(r"D:\notebooks\RetinanetTutorial")
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...


In [54]:
os.chdir(r"D:\notebooks\RetinanetTutorial/SAR Dataset/images")
data=np.empty(len(os.listdir()),dtype='object')
for i,j in zip(os.listdir(),range(len(os.listdir()))):
    data[j]="{}".format(i.strip('.png'))
print(data)

['boat0' 'boat1' 'boat10' 'boat100' 'boat101' 'boat102' 'boat103'
 'boat104' 'boat105' 'boat106' 'boat107' 'boat108' 'boat109' 'boat11'
 'boat110' 'boat111' 'boat112' 'boat113' 'boat114' 'boat115' 'boat116'
 'boat117' 'boat118' 'boat119' 'boat12' 'boat120' 'boat121' 'boat122'
 'boat123' 'boat124' 'boat125' 'boat126' 'boat127' 'boat128' 'boat129'
 'boat13' 'boat130' 'boat131' 'boat132' 'boat133' 'boat134' 'boat135'
 'boat136' 'boat137' 'boat138' 'boat139' 'boat14' 'boat140' 'boat141'
 'boat142' 'boat143' 'boat144' 'boat145' 'boat146' 'boat147' 'boat148'
 'boat149' 'boat15' 'boat150' 'boat151' 'boat152' 'boat153' 'boat154'
 'boat155' 'boat156' 'boat157' 'boat158' 'boat159' 'boat16' 'boat160'
 'boat161' 'boat162' 'boat163' 'boat164' 'boat165' 'boat166' 'boat167'
 'boat168' 'boat169' 'boat17' 'boat170' 'boat171' 'boat172' 'boat173'
 'boat174' 'boat175' 'boat176' 'boat177' 'boat178' 'boat179' 'boat18'
 'boat180' 'boat181' 'boat182' 'boat183' 'boat184' 'boat185' 'boat186'
 'boat187' 'boat188

In [66]:
os.chdir(r"D:\notebooks\RetinanetTutorial/SAR Dataset/ImageSets/Main")
f=open("trainval.txt","w+")
os.listdir()
for i in data:
    f.write("{}\n".format((i)))
f.close()

In [88]:
train,test = train_test_split(data,test_size=0.1,shuffle=True,random_state=8)
train,val = train_test_split(train,test_size=0.2,shuffle=True,random_state=8)
print('train:{}, val:{}, test:{}'.format(len(train),len(val),len(test)))

train:446, val:112, test:63


In [90]:
os.chdir(r"D:\notebooks\RetinanetTutorial/SAR Dataset/ImageSets/Main")
trainf = open("train.txt","w+")
valf = open("val.txt","w+")
testf = open("test.txt","w+")
for i in train:
    trainf.write("{}\n".format(i))
for i in val:
    valf.write("{}\n".format(i))
for i in test:
    testf.write("{}\n".format(i))
trainf.close()
valf.close()
testf.close()

In [24]:
os.chdir(r"D:\notebooks\RetinanetTutorial\SAR_Dataset\images")
for i in os.listdir():
    img_png = Image.open("D:/notebooks/RetinanetTutorial/SAR_Dataset/images/{}.png".format(i.strip('.png')))
    img_png = img_png.convert('RGB')
    img_png.save("D:/notebooks/RetinanetTutorial/SAR_Dataset/JPEGImages/{}.jpg".format(i.strip('.png')))

In [102]:
!mkdir D:\notebooks\RetinanetTutorial\Output\Snapshots

In [29]:
%%writefile pascal_voc.py
# %load pascal_voc.py
"""
Copyright 2017-2018 Fizyr (https://fizyr.com)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

from ..preprocessing.generator import Generator
from ..utils.image import read_image_bgr

import os
import numpy as np
from six import raise_from
from PIL import Image

try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

voc_classes = {
    'boat'   : 0
}


def _findNode(parent, name, debug_name=None, parse=None):
    if debug_name is None:
        debug_name = name

    result = parent.find(name)
    if result is None:
        raise ValueError('missing element \'{}\''.format(debug_name))
    if parse is not None:
        try:
            return parse(result.text)
        except ValueError as e:
            raise_from(ValueError('illegal value for \'{}\': {}'.format(debug_name, e)), None)
    return result


class PascalVocGenerator(Generator):
    """ Generate data for a Pascal VOC dataset.

    See http://host.robots.ox.ac.uk/pascal/VOC/ for more information.
    """

    def __init__(
        self,
        data_dir,
        set_name,
        classes=voc_classes,
        image_extension='.jpg',
        skip_truncated=False,
        skip_difficult=False,
        **kwargs
    ):
        """ Initialize a Pascal VOC data generator.

        Args
            base_dir: Directory w.r.t. where the files are to be searched (defaults to the directory containing the csv_data_file).
            csv_class_file: Path to the CSV classes file.
        """
        self.data_dir             = data_dir
        self.set_name             = set_name
        self.classes              = classes
        self.image_names          = [line.strip().split(None, 1)[0] for line in open(os.path.join(data_dir, 'ImageSets', 'Main', set_name + '.txt')).readlines()]
        self.image_extension      = image_extension
        self.skip_truncated       = skip_truncated
        self.skip_difficult       = skip_difficult

        self.labels = {}
        for key, value in self.classes.items():
            self.labels[value] = key

        super(PascalVocGenerator, self).__init__(**kwargs)

    def size(self):
        """ Size of the dataset.
        """
        return len(self.image_names)

    def num_classes(self):
        """ Number of classes in the dataset.
        """
        return len(self.classes)

    def has_label(self, label):
        """ Return True if label is a known label.
        """
        return label in self.labels

    def has_name(self, name):
        """ Returns True if name is a known class.
        """
        return name in self.classes

    def name_to_label(self, name):
        """ Map name to label.
        """
        return self.classes[name]

    def label_to_name(self, label):
        """ Map label to name.
        """
        return self.labels[label]

    def image_aspect_ratio(self, image_index):
        """ Compute the aspect ratio for an image with image_index.
        """
        path  = os.path.join(self.data_dir, 'JPEGImages', self.image_names[image_index] + self.image_extension)
        image = Image.open(path)
        return float(image.width) / float(image.height)

    def image_path(self, image_index):
        """ Get the path to an image.
        """
        return os.path.join(self.data_dir, 'JPEGImages', self.image_names[image_index] + self.image_extension)

    def load_image(self, image_index):
        """ Load an image at the image_index.
        """
        return read_image_bgr(self.image_path(image_index))

    def __parse_annotation(self, element):
        """ Parse an annotation given an XML element.
        """
        truncated = _findNode(element, 'truncated', parse=int)
        difficult = _findNode(element, 'difficult', parse=int)

        class_name = _findNode(element, 'name').text
        if class_name not in self.classes:
            raise ValueError('class name \'{}\' not found in classes: {}'.format(class_name, list(self.classes.keys())))

        box = np.zeros((4,))
        label = self.name_to_label(class_name)

        bndbox    = _findNode(element, 'bndbox')
        box[0] = _findNode(bndbox, 'xmin', 'bndbox.xmin', parse=float) - 1
        box[1] = _findNode(bndbox, 'ymin', 'bndbox.ymin', parse=float) - 1
        box[2] = _findNode(bndbox, 'xmax', 'bndbox.xmax', parse=float) - 1
        box[3] = _findNode(bndbox, 'ymax', 'bndbox.ymax', parse=float) - 1

        return truncated, difficult, box, label

    def __parse_annotations(self, xml_root):
        """ Parse all annotations under the xml_root.
        """
        annotations = {'labels': np.empty((len(xml_root.findall('object')),)), 'bboxes': np.empty((len(xml_root.findall('object')), 4))}
        for i, element in enumerate(xml_root.iter('object')):
            try:
                truncated, difficult, box, label = self.__parse_annotation(element)
            except ValueError as e:
                raise_from(ValueError('could not parse object #{}: {}'.format(i, e)), None)

            if truncated and self.skip_truncated:
                continue
            if difficult and self.skip_difficult:
                continue

            annotations['bboxes'][i, :] = box
            annotations['labels'][i] = label

        return annotations

    def load_annotations(self, image_index):
        """ Load annotations for an image_index.
        """
        filename = self.image_names[image_index] + '.xml'
        try:
            tree = ET.parse(os.path.join(self.data_dir, 'Annotations', filename))
            return self.__parse_annotations(tree.getroot())
        except ET.ParseError as e:
            raise_from(ValueError('invalid annotations file: {}: {}'.format(filename, e)), None)
        except ValueError as e:
            raise_from(ValueError('invalid annotations file: {}: {}'.format(filename, e)), None)


Writing pascal_voc.py


In [149]:
os.chdir(r'D:\notebooks\RetinanetTutorial/keras-retinanet')
!pip install .

Processing d:\notebooks\retinanettutorial\keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp38-cp38-win_amd64.whl size=129986 sha256=3924eb931eea524d4726def3085b216c663ef4922f18f761c3bff939b5176e44
  Stored in directory: c:\users\acer\appdata\local\pip\cache\wheels\97\7c\3f\12a1035600d9bfb44d2450baba76d4e464408cced908110512
Successfully built keras-retinanet
  Attempting uninstall: keras-retinanet
    Found existing installation: keras-retinanet 1.0.0
    Uninstalling keras-retinanet-1.0.0:
      Successfully uninstalled keras-retinanet-1.0.0


In [130]:
os.chdir(r'D:\notebooks\RetinanetTutorial/keras-retinanet')
!python setup.py build_ext --inplace

running build_ext
skipping 'keras_retinanet/utils\compute_overlap.c' Cython extension (up-to-date)
copying build\lib.win-amd64-3.8\keras_retinanet\utils\compute_overlap.cp38-win_amd64.pyd -> keras_retinanet\utils


In [15]:
os.chdir(r'D:\notebooks\RetinanetTutorial/keras-retinanet')
%run keras_retinanet\bin\train.py --tensorboard-dir D:\notebooks\RetinanetTutorial\Output --snapshot-path D:\notebooks\RetinanetTutorial\Output\Snapshots --random-transform --steps 446 pascal D:\notebooks\RetinanetTutorial\SAR_Dataset

Creating model, this may take a second...
Model: "retinanet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1 (Conv2D)                 (None, None, None,   9408        ['input_13[0][0]']               
                                64)                                                               
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, None, None,   256         ['conv1[0][0]']                  
                                64)             

D:\notebooks\RetinanetTutorial\keras-retinanet\keras_retinanet\bin\train.py:538: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  return training_model.fit_generator(


Epoch 1/50
446/446 [==============================] - ETA: 0s - loss: 2.9859 - regression_loss: 2.4113 - classification_loss: 0.5746

Running network: 100% (112 of 112) |#####| Elapsed Time: 0:00:18 Time:  0:00:18
Parsing annotations: 100% (112 of 112) |#| Elapsed Time: 0:00:00 Time:  0:00:00


223 instances of class boat with average precision: 0.2909
mAP: 0.2909

Epoch 00001: saving model to D:\notebooks\RetinanetTutorial\Output\Snapshots\resnet50_pascal_01.h5
446/446 [==============================] - 170s 365ms/step - loss: 2.9859 - regression_loss: 2.4113 - classification_loss: 0.5746 - mAP: 0.2909 - lr: 1.0000e-05
Epoch 2/50
446/446 [==============================] - ETA: 0s - loss: 2.1709 - regression_loss: 1.8073 - classification_loss: 0.3636

...

223 instances of class boat with average precision: 0.6289
mAP: 0.6289

Epoch 00028: saving model to D:\notebooks\RetinanetTutorial\Output\Snapshots\resnet50_pascal_28.h5
446/446 [==============================] - 165s 369ms/step - loss: 0.8041 - regression_loss: 0.6752 - classification_loss: 0.1288 - mAP: 0.6289 - lr: 1.0000e-07
Epoch 29/50
446/446 [==============================] - ETA: 0s - loss: 0.8054 - regression_loss: 0.6595 - classification_loss: 0.1458

Running network: 100% (112 of 112) |#####| Elapsed Time: 0:00:14 Time:  0:00:14
Parsing annotations: 100% (112 of 112) |#| Elapsed Time: 0:00:00 Time:  0:00:00


223 instances of class boat with average precision: 0.6287
mAP: 0.6287

Epoch 00029: saving model to D:\notebooks\RetinanetTutorial\Output\Snapshots\resnet50_pascal_29.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.0000000116860975e-08.
446/446 [==============================] - 164s 367ms/step - loss: 0.8054 - regression_loss: 0.6595 - classification_loss: 0.1458 - mAP: 0.6287 - lr: 1.0000e-07
Epoch 30/50
446/446 [==============================] - ETA: 0s - loss: 0.9060 - regression_loss: 0.6993 - classification_loss: 0.2067

Running network: 100% (112 of 112) |#####| Elapsed Time: 0:00:14 Time:  0:00:14
Parsing annotations: 100% (112 of 112) |#| Elapsed Time: 0:00:00 Time:  0:00:00


223 instances of class boat with average precision: 0.6255
mAP: 0.6255

Epoch 00030: saving model to D:\notebooks\RetinanetTutorial\Output\Snapshots\resnet50_pascal_30.h5
446/446 [==============================] - 166s 372ms/step - loss: 0.9060 - regression_loss: 0.6993 - classification_loss: 0.2067 - mAP: 0.6255 - lr: 1.0000e-08


In [4]:
import tensorflow.compat.v1 as tf
tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



In [4]:
import tensorflow as t
t.test.is_built_with_cuda()

True

In [31]:
!mkdir D:\notebooks\RetinanetTutorial\RetinanetModels

A subdirectory or file D:\notebooks\RetinanetTutorial\RetinanetModels already exists.


In [3]:
os.chdir(r"D:/notebooks/RetinanetTutorial/keras-retinanet")
%run keras_retinanet\bin\convert_model.py D:\notebooks\RetinanetTutorial\RetinanetModels\trained_model.h5 D:\notebooks\RetinanetTutorial\RetinanetModels\Inference_model.h5

C:\Users\acer\AppData\Roaming\Python\Python38\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [7]:
%%writefile testDetector.py
# %load testDetector.py
# Code adapted from https://github.com/fizyr/keras-retinanet
# jasperebrown@gmail.com
# 2020

# This script loads a single image, runs inferencing on it
# and saves that image back out with detections overalaid.

# You need to set the model_path and image_path below

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import preprocess_image, resize_image

# import miscellaneous modules
import cv2
import os
import numpy as np
import time
from PIL import Image

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow.compat.v1 as tf

model_path = 'D:/notebooks/RetinanetTutorial/RetinanetModels/Inference_model.h5'
image_path = 'D:/notebooks/s1.jpg'
image_output_path = 'D:/notebooks/RetinanetTutorial/SAR_Dataset/detected_images/s2_detected.jpg'
confidence_cutoff = 0.5 #Detections below this confidence will be ignored

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
tf.compat.v1.keras.backend.set_session(get_session())

# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
#model_path = os.path.join('..', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

print("Loading image from {}".format(image_path))
image = np.asarray(Image.open(image_path).convert('RGB'))
image = image[:, :, ::-1].copy()

# load retinanet model
print("Loading Model: {}".format(model_path))
model = models.load_model(model_path, backbone_name='resnet50')

#Check that it's been converted to an inference model
try:
    model = models.convert_model(model)
except:
    print("Model is likely already an inference model")

# load label to names mapping for visualization purposes
labels_to_names = {0: 'anamoly'}

# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# Image formatting specific to Retinanet
image = preprocess_image(image)
image, scale = resize_image(image)

# Run the inference
start = time.time()

boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < confidence_cutoff:
        break

    #Add boxes and captions
    color = (255, 255, 255)
    thickness = 2
    b = np.array(box).astype(int)
    cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), color, thickness, cv2.LINE_AA)

    if(label > len(labels_to_names)):
        print("WARNING: Got unknown label, using 'detection' instead")
        caption = "Detection {:.3f}".format(score)
    else:
        caption = "{} {:.3f}".format(labels_to_names[label], score)

    cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

#Write out image
draw = Image.fromarray(draw)
draw.save(image_output_path)


Overwriting testDetector.py


In [8]:
os.chdir(r"D:/notebooks/RetinanetTutorial/keras-retinanet")
%run testDetector.py

Loading image from D:/notebooks/s1.jpg
Loading Model: D:/notebooks/RetinanetTutorial/RetinanetModels/Inference_model.h5
Model is likely already an inference model
processing time:  3.8536810874938965
